
# Myocardial Infarction Complications — Klasyfikacja
**Zbiór danych:** Kaggle — *Myocardial Infarction Complications* (CSV)

W tym notatniku zrobisz krok po kroku:
1. Wczytanie i szybki przegląd danych  
2. Automatyczna detekcja kolumny celu (z możliwością ręcznej zmiany)  
3. Podział train/test (stratyfikacja) i przygotowanie danych  
4. Modele: **Drzewo decyzyjne**, **Bagging**, **Random Forest**, **XGBoost**  
5. Ewaluacja: **accuracy, precision, recall, F1, ROC-AUC**, **macierze pomyłek**, **krzywe ROC/PR**  
6. **Feature importance** dla RF/XGB oraz krótkie wnioski

> Jeśli `xgboost` nie jest zainstalowany, pokażemy komunikat i pominiesz ten model (reszta działa).


In [ ]:

# (opcjonalnie) Zainstaluj brakujące biblioteki lokalnie
# !pip install -q pandas numpy scikit-learn matplotlib xgboost


In [ ]:

import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, RocCurveDisplay, PrecisionRecallDisplay,
    ConfusionMatrixDisplay, classification_report
)
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

# XGBoost (opcjonalnie)
try:
    from xgboost import XGBClassifier
    HAS_XGB = True
except Exception:
    HAS_XGB = False

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)



## 1) Wczytanie danych
Umieść pobrany z Kaggle plik CSV (np. `myocardial_infarction_complications.csv`) **w tym samym folderze co notatnik**.
Poniższy kod spróbuje automatycznie znaleźć plik i kolumnę celu.


In [ ]:

# Kandydaci ścieżek do pliku — dodaj tu własną nazwę, jeśli inna
DATA_CANDIDATES = [
    'myocardial_infarction_complications.csv',
    'myocardial-infarction-complications.csv',
    'mi_complications.csv',
    'data.csv',
    './data/myocardial_infarction_complications.csv',
    '/mnt/data/myocardial_infarction_complications.csv'
]

csv_path = None
for p in DATA_CANDIDATES:
    if os.path.exists(p):
        csv_path = p
        break

if csv_path is None:
    raise FileNotFoundError(
        "Nie znaleziono pliku CSV. Zmień listę DATA_CANDIDATES powyżej albo dodaj plik obok notatnika."
    )

df = pd.read_csv(csv_path)
print(f"Wczytano dane z: {csv_path} — shape={df.shape}")
display(df.head())



### 1.1 Automatyczna detekcja kolumny celu
Notatnik spróbuje znaleźć kolumnę celu (np. *complication*, *outcome*, *death*, *mortality*, *readmission* itp.).
Jeśli wykrycie się nie powiedzie — **ustaw ręcznie nazwę w zmiennej `TARGET`**.


In [ ]:

# Spróbuj znaleźć target po nazwie
LOWER_COLS = {c.lower(): c for c in df.columns}
PATTERNS = [
    r'complic', r'outcome', r'target', r'class', r'label', r'death', r'mortal', r'event', r'adverse'
]

found = None
for pat in PATTERNS:
    for lc, orig in LOWER_COLS.items():
        if re.search(pat, lc):
            found = orig
            break
    if found:
        break

# Jeśli nie znalazło, użyj ostatniej kolumny jako domysł (częsta konwencja w niektórych CSV)
TARGET = found if found is not None else df.columns[-1]
print("Wykryta/założona kolumna celu:", TARGET)
print("Unikalne wartości celu:", pd.Series(df[TARGET]).value_counts(dropna=False).to_dict())

# Jeśli chcesz ręcznie: odkomentuj i podaj nazwę
# TARGET = "<TU_WPROWADŹ_NAZWĘ_KOLUMNY_CELU>"



## 2) Szybki podgląd i czyszczenie
Sprawdzimy typy kolumn, braki danych i podstawowe statystyki.


In [ ]:

print("Typy kolumn:")
display(df.dtypes)

print("\nBraki danych w kolumnach:")
display(df.isna().sum().sort_values(ascending=False))

display(df.describe(include='all').transpose().head(20))



## 3) Podział na zbiory i preprocessing
- Stratyfikacja względem klasy
- Imputacja braków: medianą (num) / najczęstszą (cat)
- One-hot encoding dla kategorii (drzewa i RF nie wymagają skalowania, ale pipeline jest ogólny)
- **Uwaga na nierównowagę klas** — ustawimy `class_weight='balanced'` w modelach drzewiastych.


In [ ]:

# Oddziel X/y
y_raw = df[TARGET]
X = df.drop(columns=[TARGET])

# Spróbuj konwersji celu na int/kat.
if pd.api.types.is_numeric_dtype(y_raw):
    # Jeśli wartości nie są 0/1, spróbuj zbinarnizować gdy są tylko 2 unikalne wartości
    uniques = np.unique(y_raw.dropna())
    if len(uniques) == 2 and set(uniques) != {0,1}:
        mapping = {uniques[0]:0, uniques[1]:1}
        y = y_raw.map(mapping).astype(int)
    else:
        y = y_raw.astype(int) if y_raw.dropna().astype(int).equals(y_raw.dropna()) else y_raw.astype(float)
else:
    y = y_raw.astype('category')

# Podział:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y if len(pd.Series(y).unique())>1 else None, random_state=RANDOM_STATE
)
print("Rozmiary:", X_train.shape, X_test.shape)

# Wykryj typy kolumn
num_cols = [c for c in X.columns if pd.api.types.is_numeric_dtype(X[c])]
cat_cols = [c for c in X.columns if c not in num_cols]

numeric_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    # Skalowanie nie jest krytyczne dla drzew, ale zostawiamy dla ogólności (może posłużyć XGB)
    ('scaler', StandardScaler(with_mean=False) if len(cat_cols)==0 else StandardScaler())
])

categorical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_pipe, num_cols),
        ('cat', categorical_pipe, cat_cols)
    ],
    remainder='drop'
)

print(f"Liczba cech numerycznych: {len(num_cols)}, kategorycznych: {len(cat_cols)}")


In [ ]:

def evaluate_model(name, model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    proba_available = hasattr(model, "predict_proba")
    y_proba = model.predict_proba(X_test)[:,1] if proba_available and len(np.unique(y_test))==2 else None

    # Dla klasyfikacji wieloklasowej użyj macro-averages
    average = 'binary' if len(np.unique(y_test))==2 else 'macro'

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average=average, zero_division=0)
    rec = recall_score(y_test, y_pred, average=average, zero_division=0)
    f1 = f1_score(y_test, y_pred, average=average, zero_division=0)
    roc = roc_auc_score(y_test, y_proba) if y_proba is not None else None

    print(f"""[{name}] 
Accuracy:  {acc:.4f}
Precision: {prec:.4f}
Recall:    {rec:.4f}
F1-score:  {f1:.4f}
ROC-AUC:   {roc:.4f} if roc is not None else '—'""")

    # Macierz pomyłek
    cm = confusion_matrix(y_test, y_pred)
    ConfusionMatrixDisplay(confusion_matrix=cm).plot()
    plt.title(f"Confusion Matrix — {name}")
    plt.show()

    # ROC & PR (jeśli binarny i mamy proby)
    if y_proba is not None:
        RocCurveDisplay.from_predictions(y_test, y_proba)
        plt.title(f"ROC Curve — {name}")
        plt.show()

        PrecisionRecallDisplay.from_predictions(y_test, y_proba)
        plt.title(f"Precision-Recall Curve — {name}")
        plt.show()

    return {
        'model': name,
        'accuracy': float(acc),
        'precision': float(prec),
        'recall': float(rec),
        'f1': float(f1),
        'roc_auc': float(roc) if roc is not None else None
    }



## 4) Modele
### 4.1 Drzewo decyzyjne (baseline)
Ustawimy `max_depth=3` (zgodnie z tutorialem) i `class_weight='balanced'` (na wypadek nierównowagi).


In [ ]:

tree_pipe = Pipeline([
    ('prep', preprocess),
    ('clf', DecisionTreeClassifier(max_depth=3, class_weight='balanced', random_state=RANDOM_STATE))
])
results = []
results.append(evaluate_model("Decision Tree (depth=3)", tree_pipe, X_train, X_test, y_train, y_test))



### 4.2 Bagging (na bazie drzewa)


In [ ]:

base_tree = DecisionTreeClassifier(class_weight='balanced', random_state=RANDOM_STATE)
bag_pipe = Pipeline([
    ('prep', preprocess),
    ('clf', BaggingClassifier(
        estimator=base_tree,
        n_estimators=200,
        random_state=RANDOM_STATE,
        n_jobs=-1
    ))
])
results.append(evaluate_model("Bagging (DecisionTree x200)", bag_pipe, X_train, X_test, y_train, y_test))



### 4.3 Random Forest
Również z `class_weight='balanced'`, 200 drzew i maksymalną głębokością 3.


In [ ]:

rf_pipe = Pipeline([
    ('prep', preprocess),
    ('clf', RandomForestClassifier(
        n_estimators=200, max_depth=3, class_weight='balanced',
        random_state=RANDOM_STATE, n_jobs=-1
    ))
])
results.append(evaluate_model("Random Forest (200, depth=3, balanced)", rf_pipe, X_train, X_test, y_train, y_test))



### 4.4 XGBoost (Boosting) — jeśli dostępny


In [ ]:

if HAS_XGB:
    xgb_pipe = Pipeline([
        ('prep', preprocess),
        ('clf', XGBClassifier(
            n_estimators=400, max_depth=3, learning_rate=0.1,
            subsample=1.0, colsample_bytree=1.0,
            eval_metric='logloss',
            scale_pos_weight=None,  # alternatywnie: oblicz wg nierównowagi
            random_state=RANDOM_STATE, n_jobs=-1, tree_method='hist'
        ))
    ])
    results.append(evaluate_model("XGBoost (400, depth=3)", xgb_pipe, X_train, X_test, y_train, y_test))
else:
    print("Brak biblioteki xgboost — zainstaluj i uruchom ponownie, by uwzględnić boosting.")



## 5) Ważność cech
Pokażemy ranking cech dla Random Forest (i XGBoost, jeśli dostępny).


In [ ]:

def plot_feature_importances(fitted_pipe, title):
    clf = fitted_pipe.named_steps['clf']
    prep = fitted_pipe.named_steps['prep']

    # Nazwy kolumn po przetwarzaniu
    num_cols = prep.transformers_[0][2]
    cat_cols = prep.transformers_[1][2] if len(prep.transformers_)>1 else []
    ohe = None
    try:
        ohe = prep.named_transformers_['cat'].named_steps.get('onehot')
    except Exception:
        pass

    if ohe is not None and hasattr(ohe, "get_feature_names_out"):
        cat_names = list(ohe.get_feature_names_out(cat_cols))
    else:
        cat_names = list(cat_cols)

    feature_names = list(num_cols) + cat_names

    if hasattr(clf, "feature_importances_"):
        importances = clf.feature_importances_
        order = np.argsort(importances)[::-1][:30]
        plt.figure()
        plt.bar(range(len(order)), importances[order])
        plt.xticks(range(len(order)), [feature_names[i] if i < len(feature_names) else f"f{i}" for i in order], rotation=90)
        plt.title(title)
        plt.tight_layout()
        plt.show()
    else:
        print("Model nie udostępnia feature_importances_.")

# Fit na train, potem wykresy
rf_pipe.fit(X_train, y_train)
plot_feature_importances(rf_pipe, "Feature Importance — Random Forest")

if HAS_XGB:
    xgb_pipe.fit(X_train, y_train)
    plot_feature_importances(xgb_pipe, "Feature Importance — XGBoost")



## 6) Porównanie modeli i (opcjonalnie) walidacja krzyżowa


In [ ]:

results_df = pd.DataFrame(results)
display(results_df.sort_values('f1', ascending=False))

# (Opcjonalnie) CV dla RF
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
rf_cv = Pipeline([('prep', preprocess), ('clf', RandomForestClassifier(n_estimators=200, max_depth=3, class_weight='balanced', random_state=RANDOM_STATE, n_jobs=-1))])
cv_scores = cross_val_score(rf_cv, X, y, scoring='f1' if len(np.unique(y))==2 else 'f1_macro', cv=cv, n_jobs=-1)
print("RF 5-fold CV F1:", cv_scores.mean().round(4), "+/-", cv_scores.std().round(4))



## 7) Wnioski
- Drzewo (głębokość 3) to punkt odniesienia — proste i interpretowalne.  
- Bagging/Random Forest zwykle poprawiają stabilność i dokładność redukując wariancję.  
- Boosting (XGBoost) często osiąga najlepsze wyniki, ale jest wrażliwy na parametry i może przetrenować.  
- Sprawdź **feature importance** i porównaj z wiedzą dziedzinową.  
- Jeśli klasy są nierówne, patrz także na **F1** i **PR-curve**, nie tylko na accuracy.
